In [5]:

import sys
import os
from pathlib import Path
#import instructor
from pydantic import BaseModel
sys.path.insert(0, '../libs')
# Import SimpleLLMAgent from libs directory
from pydantic import BaseModel, Field
import json
import io
# Import our general LLM factory
from llm_factory_openai import LLMAgent
#from utils import download_hf_model
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv('../.env')
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please check your .env file.")


#### Test call closed source model api

In [10]:

agent = LLMAgent(model='gpt-5-nano',
                temperature=1,
                api_key=os.getenv("OPENAI_API_KEY"))
result = agent.test_connection()
print(f"OpenAI test result: {result}")

OpenAI test result: Hi there! Yes, I can respond. How can I help you today? I can explain concepts, answer questions, help with writing, coding, math, planning, brainstorming, translations, and more. If you have a task, share some details and any constraints. For example, you could ask me to explain something simply, draft an email, debug code, plan a trip, or summarize an article. What would you like to do?


In [11]:
messages = [{"role": "user", "content": 'hi'}]
response = agent.get_response_content(messages)
print(response)

Hi! Nice to meet you. What would you like to do today? I can help with explanations, writing, brainstorming ideas, coding, planning, or answering questions. Tell me what you need or drop a topic you’re curious about.


In [12]:
news_article = """
WASHINGTON (Reuters) -The U.S. will make it more difficult for global chipmakers Samsung, SK Hynix and Intel to produce chips in China by revoking permission for the companies to receive American semiconductor manufacturing equipment there, according to the Federal Register.
The U.S. Commerce Department had granted the authorizations for the companies to make chips in China as exceptions to sweeping restrictions on the sale of semiconductor equipment to China in 2022.
The companies will now need to obtain licenses to buy the equipment in China.
The licensing change will likely reduce sales to China by U.S. equipment makers KLA Corp, Lam Research and Applied Materials. The move may help domestic Chinese equipment makers, whose tools can fill gaps. They also may help Micron, a major U.S. competitor to South Korea's Samsung and SK Hynix in the memory chip sector. The authorizations will not take effect for 120 days, according to the posting.
Intel sold its Dalian NAND memory manufacturing facility in China to Seoul-based SK Hynix several years ago, but continued to manufacture wafers into 2025.
"""

system_prompt = """ 
You are an economist and trade/industrial-policy expert at the International Monetary Fund (IMF). 
Your task is to determine the type of products that are targeted by policy interventions. 

The categories to consider include:

* Medical Products: Includes medical consumables, vaccines, medicines, and medical equipment.
* Military/Civilian Dual-Use Products: Items usable for both civilian and military applications. This includes certain machinery, hydrogen, or aerospace components used in both domains.
* Low-Carbon Technology: Includes wind turbines, solar panels, biomass systems, carbon capture equipment, and other technologies contributing to the low-carbon transition.
* Semiconductors: Semiconductors and related materials or technologies that enable semiconductor development.
* Advanced Technology Products: Products that rely on cutting-edge technologies, including aerospace, opto-electronics, robotics, quantum, and nuclear technologies.
* Critical Minerals: Minerals essential for manufacturing modern technology (e.g., lithium, cobalt, rare earths).
* Steel and Aluminum: Includes crude or processed steel, iron, and aluminum products.
* Chemicals: Includes basic and specialty chemicals such as petrochemicals, fertilizers, industrial compounds, and plastics.
* Other: Use this when the sentence does not clearly fall into the categories above.

**Process:**
You will be given a list of sentences from a news article. 
1) Please read each sentence carefully and determine the most appropriate product category based on its content.
2) Output only a JSON array of objects, where each object contains:  
   - "sentence_id": integer (starting from 1)  
   - "tags": list of product categories  

**Rules:**
- More than one category may apply; list them all in "tags".  
- Always output in English.  
- Do NOT include the raw sentence text in the output. only return sentence id 
- Return only valid JSON.  

Example output format:
[
  {"sentence_id": 1, "tags": ["Semiconductors"]},
  {"sentence_id": 2, "tags": ["Semiconductors", "Advanced Technology Products"]}
]

"""

In [15]:
sentences = [sentence.strip() for sentence in news_article.split(". ") if sentence]
sentences_with_id = '\n'.join(["id {}: {}".format(i+1,s) for i, s in enumerate(sentences)])

In [17]:
print(sentences_with_id)


id 1: WASHINGTON (Reuters) -The U.S
id 2: will make it more difficult for global chipmakers Samsung, SK Hynix and Intel to produce chips in China by revoking permission for the companies to receive American semiconductor manufacturing equipment there, according to the Federal Register.
The U.S
id 3: Commerce Department had granted the authorizations for the companies to make chips in China as exceptions to sweeping restrictions on the sale of semiconductor equipment to China in 2022.
The companies will now need to obtain licenses to buy the equipment in China.
The licensing change will likely reduce sales to China by U.S
id 4: equipment makers KLA Corp, Lam Research and Applied Materials
id 5: The move may help domestic Chinese equipment makers, whose tools can fill gaps
id 6: They also may help Micron, a major U.S
id 7: competitor to South Korea's Samsung and SK Hynix in the memory chip sector
id 8: The authorizations will not take effect for 120 days, according to the posting.
Intel 

In [18]:
messages = [{"role": "system", "content": system_prompt},
            {"role": "user", "content": sentences_with_id}]
response = agent.get_response_content(messages)
print(response)

[
  {"sentence_id": 1, "tags": ["Semiconductors"]},
  {"sentence_id": 2, "tags": ["Semiconductors"]},
  {"sentence_id": 3, "tags": ["Semiconductors"]},
  {"sentence_id": 4, "tags": ["Semiconductors"]},
  {"sentence_id": 5, "tags": ["Semiconductors"]},
  {"sentence_id": 6, "tags": ["Semiconductors"]},
  {"sentence_id": 7, "tags": ["Semiconductors"]},
  {"sentence_id": 8, "tags": ["Semiconductors"]}
]
